# Create a custom-shape cutout

Create a slab with a complex cutout pattern.
By default, this notebook generates a structure of etching pattern:
- Two vertical side walls
- Two parallel channels at the top
- A deeper central cavity

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Material (in the outer runtime) before running the notebook.
2. Set structural parameters in cell 1.2 below
4. Click "Run" > "Run All" to create the structure
5. Scroll down to view the resulting structure from multiple angles

## Resulting structure:
<img src="https://i.imgur.com/j9YDJ1X.png">

## 1. Prepare the Environment
### 1.1. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("")

### 1.2. Set up structure parameters 

In [ ]:
# Slab parameters
MILLER_INDICES = (0, 0, 1)
THICKNESS = 6  # unit cell layers
VACUUM = 10.0  # Angstrom
XY_SUPERCELL_MATRIX = [[10, 0], [0, 10]]

# Custom shape parameters in crystal units of the slab
LEFT_WALL_POSITION = 0.2
RIGHT_WALL_POSITION = 0.8
LEFT_CHANNEL_POSITION = 0.25
RIGHT_CHANNEL_POSITION = 0.75
CHANNEL_TOP_Z = 0.8
CAVITY_TOP_Z = 0.5

NAME = "Custom Shape - Etching Pattern"

### 1.3. Create a condition on coordinates


In [ ]:
from typing import List

from mat3ra.made.tools.utils.coordinate import CoordinateCondition


# Example of a custom coordinate condition. Adapt coordinate conditions to your needs.
class CustomCoordinateCondition(CoordinateCondition):
    """
    Defines an etched pattern structure with side walls, channels, and a central cavity.
    All dimensions are specified as fractions of the total size (0.0 to 1.0).
    """
    left_wall_position: float = 0.2
    right_wall_position: float = 0.8
    left_channel_position: float = 0.25
    right_channel_position: float = 0.75
    channel_top_z: float = 0.8
    cavity_top_z: float = 0.5

    def condition(self, coordinate: List[float]) -> bool:
        """
        Determines which atoms to filter based on the coordinate.
        Returns True for positions where atoms should be removed.
        
        The structure consists of:
        1. Side walls: Two vertical walls running along the y-axis
        2. Channels: Two trenches at the top running along the x-axis
        3. Cavity: A deeper central region between the channels
        """
        x, y, z = coordinate

        # Define the structural elements
        is_in_left_wall = x <= self.left_wall_position
        is_in_right_wall = x >= self.right_wall_position

        is_in_left_channel = y <= self.left_channel_position
        is_in_right_channel = y >= self.right_channel_position

        is_in_channel = (
                z < self.channel_top_z and
                (is_in_left_channel or is_in_right_channel)
        )

        is_below_cavity = (
                z < self.cavity_top_z and  # Cavity depth
                self.left_wall_position < x < self.right_wall_position and  # Between walls
                not is_in_left_channel and not is_in_right_channel  # Between channels
        )

        return (
                is_in_left_wall or
                is_in_right_wall or
                is_in_channel or
                is_below_cavity
        )


condition = CustomCoordinateCondition(
    left_wall_position=LEFT_WALL_POSITION,
    right_wall_position=RIGHT_WALL_POSITION,
    left_channel_position=LEFT_CHANNEL_POSITION,
    right_channel_position=RIGHT_CHANNEL_POSITION,
    channel_depth=CHANNEL_TOP_Z,
    cavity_depth=CAVITY_TOP_Z,
).condition

### 1.3. Get input materials

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())
material = materials[0]

### 1.4. Preview the material

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize(material, repetitions=[1, 1, 1], rotation="0x")
visualize(material, repetitions=[1, 1, 1], rotation="-90x")

## 2. Create target material
### 2.1. Create a slab and apply the cutout condition

In [ ]:
from mat3ra.made.tools.modify import filter_by_condition_on_coordinates, add_vacuum
from mat3ra.made.tools.build.slab import create_slab, SlabConfiguration

slab_config = SlabConfiguration(
    bulk=material,
    miller_indices=MILLER_INDICES,
    thickness=THICKNESS,
    vacuum=0,
    xy_supercell_matrix=XY_SUPERCELL_MATRIX,
    use_orthogonal_z=True,
    use_conventional_cell=True,
)

slab_unit_cell = create_slab(slab_config)
resulting_material = filter_by_condition_on_coordinates(slab_unit_cell, condition)
resulting_material = add_vacuum(resulting_material, VACUUM)

## 3. Visualize the result

In [ ]:
visualize(resulting_material, repetitions=[1, 1, 1], rotation="0x")
visualize(resulting_material, repetitions=[1, 1, 1], rotation="-90x")
visualize(resulting_material, repetitions=[1, 1, 1], rotation="-90x,-90y")

# 4. Pass material to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials

resulting_material.name = NAME
set_materials(resulting_material)